In [9]:
# autoreload to reload modules when they change
%load_ext autoreload
%autoreload 2

import rollout
from torch import optim
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/ehu7/Projects/TransformerCircuits/src/rollout/dataset.py:36: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert( num_tokens > sequence_length, "num_tokens must be greater than sequence_length")
/Users/ehu7/Projects/TransformerCircuits/src/rollout/dataset.py:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(n_back < sequence_length-1, "n_back must be less than sequence_length-1")


In [20]:
from itertools import permutations

In [10]:
d_model = 128
n_heads = 1
n_attn_layers = 2
n_tokens = 10
max_positions = 6


dataset = rollout.dataset.ValueDataset(n_tokens, max_positions)


simpleModel = rollout.models.FlexibleTransformer(d_model, n_tokens, max_positions, n_heads, n_attn_layers)
optimizer = optim.AdamW(simpleModel.parameters(), lr=0.001)
criterion = nn.functional.cross_entropy

simple_train_losses, simple_test_losses = rollout.models.optimize_model(simpleModel, criterion, optimizer, dataset, n_epochs=500, batch_size=1024)

Epoch 0: Train Loss: 2.4328184127807617, Test Loss: 1.907546043395996
Epoch 100: Train Loss: 0.00023883560788817704, Test Loss: 0.0003346828743815422
Epoch 200: Train Loss: 6.630298594245687e-05, Test Loss: 7.699814159423113e-05
Epoch 300: Train Loss: 4.117744174436666e-05, Test Loss: 3.5974153433926404e-05
Epoch 400: Train Loss: 2.969959496113006e-05, Test Loss: 2.5192508473992348e-05
